In [1]:
import chess

In [2]:
board = chess.Board()

In [3]:
import time
from IPython.display import display, HTML, clear_output

In [4]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [5]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [6]:
def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        planilla = []
        numero_jugada = 1.0
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
            planilla.append(str(numero_jugada) + '. ' + uci)
            print(planilla)
            numero_jugada = numero_jugada + 0.5 
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)

In [7]:
import random

In [8]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

In [9]:
def human_player(board):
    display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

In [10]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

In [11]:
def player1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis1(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [12]:
def staticAnalysis1(board, move, my_color):
    score = 0
    ## Check some things about this move:
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [13]:
def player2(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis2(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [14]:
def staticAnalysis2(board, move, my_color):                              # mejora de la funcion de analisis anterior
    score = random.random()
    ## Check some things about this move:
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [15]:
def player3(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis3(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [16]:
def staticAnalysis3(board, move, my_color):                             # mejora de la funcion de analisis anterior
    ## Check some things about this move:
    score = random.random()
    score += 1 if board.is_capture(move) else 0
    score += 1 if board.is_castling(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 3), 
                           (chess.KING, 0),
                           (chess.QUEEN, 9), 
                           (chess.KNIGHT, 3),
                           (chess.ROOK, 5)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    score -= 100 if board.is_stalemate() else 0
    score -= 0 if board.has_castling_rights(my_color) else 1
    score += 0 if board.has_castling_rights(not my_color) else 1
    return score

In [17]:
def player4(board):
    score_and_move = minimax(board, 3)
    move = score_and_move[1]
    return move.uci()

In [18]:
def minimax(board, depth):
#Returns a tuple (score, bestmove) for the position at the given depth
    if depth == 0 or board.is_checkmate() or board.is_stalemate() or board.is_fivefold_repetition() or board.is_insufficient_material() or board.can_claim_draw():
        return [staticAnalysis4(board), None]
    else: 
        if board.turn == chess.WHITE:
            bestscore = -float("inf")
            bestmove = None
            for move in list(board.legal_moves):
                newboard = board.copy()
                newboard.push(move)
                score_and_move = minimax(newboard, depth - 1)
                score = score_and_move[0]
                if score > bestscore: # white maximizes her score
                    bestscore = score
                    bestmove = move
            return [bestscore, bestmove]
        else:
            bestscore = float("inf")
            bestmove = None
            for move in list(board.legal_moves):
                newboard = board.copy()
                newboard.push(move)
                score_and_move = minimax(newboard, depth - 1)
                score = score_and_move[0]
                if score < bestscore: # black minimizes his score
                    bestscore = score
                    bestmove = move
            return [bestscore, bestmove]

In [19]:
def staticAnalysis4(board):
    score = random.random()
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 3), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 9), 
                           (chess.KNIGHT, 3),
                           (chess.ROOK, 5)]:
        score += len(board.pieces(piece, chess.WHITE)) * value
        score -= len(board.pieces(piece, chess.BLACK)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    if board.turn == chess.BLACK and board.is_checkmate():
        score += 100
    if board.turn == chess.WHITE and board.is_checkmate():
        score -= 100
    return score

In [20]:
def player5(board):
    score_and_move = alphabeta(board, 3, -float("inf"), float("inf"))
    move = score_and_move[1]
    return move.uci()

In [21]:
def alphabeta(board, depth, alpha, beta):
#Returns a tuple (score, bestmove) for the position at the given depth
    if depth == 0 or board.is_checkmate() or board.is_stalemate() or board.is_fivefold_repetition() or board.is_insufficient_material() or board.can_claim_draw():
        return [staticAnalysis5(board), None]
    else: 
        if board.turn == chess.WHITE:
            bestmove = None
            for move in board.legal_moves:
                newboard = board.copy()
                newboard.push(move)
                score_and_move = alphabeta(newboard, depth - 1, alpha, beta)
                score = score_and_move[0]
                if score > alpha: # white maximizes her score
                    alpha = score
                    bestmove = move
                    if alpha >= beta: # alpha-beta cutoff
                        break
            return [alpha, bestmove]
        else:
            bestmove = None
            for move in board.legal_moves:
                newboard = board.copy()
                newboard.push(move)
                score_and_move = alphabeta(newboard, depth - 1, alpha, beta)
                score = score_and_move[0]
                if score < beta: # black minimizes his score
                    beta = score
                    bestmove = move
                    if alpha >= beta: # alpha-beta cutoff
                        break
            return [beta, bestmove]

In [22]:
def staticAnalysis5(board):
    score = random.random()
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 3), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 9), 
                           (chess.KNIGHT, 3),
                           (chess.ROOK, 5)]:
        score += len(board.pieces(piece, chess.WHITE)) * value
        score -= len(board.pieces(piece, chess.BLACK)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    if board.turn == chess.BLACK and board.is_checkmate():
        score += 100
    if board.turn == chess.WHITE and board.is_checkmate():
        score -= 100
    return score

In [23]:
first_game = play_game(player4,player3)

['1.0. d2d4', '1.5. c7c5', '2.0. d4c5', '2.5. e7e6', '3.0. b2b4', '3.5. f8c5', '4.0. b4c5', '4.5. e6e5', '5.0. c2c3', '5.5. a7a6', '6.0. d1d6', '6.5. d8g5', '7.0. c1g5', '7.5. b7b5', '8.0. d6e5', '8.5. g8e7', '9.0. e5e7']
checkmate: White wins!


In [24]:
2+2

4